In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

TODO:

when running with cloud. 
if you dont need to keep the execution logs, change the cloud settings to keep it for a short while
Check that the cache if you need to set the cache to false (especially in plot process)

In [2]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [3]:
import os
json_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"hai-gcp-genomic-beb9207a40b8.json")
!ls $json_path

/home/eila/git_igvf/atomic-workflows/nextflow/hai-gcp-genomic-beb9207a40b8.json


In [4]:
# # in the command line. same env
import os
json_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"hai-gcp-genomic-beb9207a40b8.json")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_path
!gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

Activated service account credentials for: [eila-batch@hai-gcp-genomic.iam.gserviceaccount.com]


In [5]:
!gcloud auth list
project_name = !gcloud config get-value project
project_name = project_name[0]
project_name
# gcloud config set account

                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com
        eila@stanford.edu

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



'hai-gcp-genomic'

In [6]:
import os
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [7]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [8]:
import os
bin_fld = os.path.join(os.getcwd(),"bin")
bin_fld

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin'

In [9]:
!cp /home/eila/git_igvf/atomic-workflows/bin_files/* /home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin/


In [10]:
!chmod +x {os.path.join(bin_fld, "joint_cell_plotting.py")}
!chmod +x {os.path.join(bin_fld, "joint_cell_plotting_density.R")}
!chmod +x {os.path.join(bin_fld, "write_html.py")}
# !chmod +x {os.path.join(bin_fld, "create_list.sh")}

In [11]:
# in the command line. based on:
# https://nextflow.io/blog/2023/nextflow-with-gbatch.html
# gcloud services enable batch.googleapis.com compute.googleapis.com logging.googleapis.com
# gcloud iam service-accounts create eila-batch
# gcloud iam service-accounts list 


# gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/storage.admin"


In [12]:
# !nextflow -self-update

In [13]:
!nextflow info

  Version: 23.10.1 build 5891
  Created: 12-01-2024 22:01 UTC 
  System: Linux 5.10.0-28-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.22+7-post-Debian-1deb11u1
  Encoding: UTF-8 (UTF-8)



In [14]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Navigate two folders above
two_folders_above = os.path.abspath(os.path.join(current_directory, '..', '..'))

nf_data_base_path = os.path.join(two_folders_above,'nf_data')
# Print the result
print(nf_data_base_path)


/home/eila/git_igvf/atomic-workflows/nextflow/nf_data


In [15]:
nf_data_base_path

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_data'

In [16]:
# nf_data_project="test_atac_pipeline"
nf_data_project="test_joint_pipeline"

In [17]:
!cp ../../config_files/docker-google-batch.config ../workflow_fg_single_cell_joint/conf/

In [18]:

# Write the configuration file
with open("nextflow_joint.config", "w") as config_file:
    config_file.write(f"""
manifest {{
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_joint.nf'
}}

profiles {{
  google_batch_profile {{
    workDir = 'gs://eila-nextflow-bucket/work/JOINT'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = '{project_name}'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }}
}}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_PY='joint_cell_plotting.py'

//Minimum number of UMIs/fragments required for a cell to be plotted
params.SC_JOINT_CELL_BARCODE_PLOT_REMOVE_LOW_YIELDING_CELLS=10
params.SC_JOINT_CELL_BARCODE_PLOT_METADATA_FILES='{nf_data_base_path}/{nf_data_project}/vm_joint_files.tsv'
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_UMIS=100
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_GENES=200
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_TSS=4

//Cutoff for minimum number of ATAC fragments
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_FRAGS=100
params.SC_JOINT_CELL_BARCODE_PLOT_PKR='BMMC-single-donor'
params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_R='joint_cell_plotting_density.R'

//params.SC_JOINT_CELL_BARCODE_PLOT_HTML_LIST_SCRIPT='create_list.sh'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_SCRIPT='write_html.py'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_FILES='{nf_data_base_path}/{nf_data_project}/vm_joint_html.tsv'



""")


In [19]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [20]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/work
# !gsutil rm -r gs://eila-nextflow-bucket/work/JOINT

In [21]:
# !rm .nextflow*

In [22]:
# !ls nextflow.config

In [23]:
!nextflow run nf_wf_fg_singlecell_joint.nf -c nextflow_joint.config -profile google_batch_profile -with-tower -resume 


# !nextflow run nf_wf_fg_singlecell_joint.nf -c nextflow_joint.config -profile google_batch_profile -with-tower -resume

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile


# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


N E X T F L O W  ~  version 23.10.1
Launching `nf_wf_fg_singlecell_joint.nf` [kickass_hugle] DSL2 - revision: cc5ee23a93
Before calling run_joint_barcode_cell_plotting
[-        ] process > run_joint_barcode_cell_plot... -
[-        ] process > run_joint_cell_plotting_den... -
[-        ] process > run_joint_metrics_html_report  -
After calling run_joint_barcode_cell_plotting
Before calling run_joint_cell_plotting_density
After calling run_joint_cell_plotting_density
Before run_joint_metrics_html_report
After run_joint_metrics_html_report

Monitor the execution with Nextflow Tower using this URL: https://tower.nf/orgs/ort_eila_org/workspaces/ort_eila_wrkspc/watch/2THExQVsV0zhfU
[-        ] process > run_joint_barcode_cell_plot... -
[-        ] process > run_joint_cell_plotting_den... -
[-        ] process > run_joint_metrics_html_report  -
After calling run_joint_barcode_cell_plotting
Before calling run_joint_cell_plotting_density
After calling run_joint_cell_plotting_density
Before ru

In [24]:
!cat .nextflow.log

Apr-02 03:04:30.600 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_joint.nf -c nextflow_joint.config -profile google_batch_profile -with-tower -resume
Apr-02 03:04:30.848 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.1
Apr-02 03:04:30.883 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.3,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.1
Apr-02 03:04:30.900 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Apr-02 03:04:30.902 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Apr-02 03:04:30.908 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Apr-02 03:04:30.925 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Apr-02 03:04:30.953 [main] DEBUG nextflow.con

In [25]:
# nf_data_base_path = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/'

# # Write the configuration file
# with open("nextflow_vm.config", "w") as config_file:
#     config_file.write(f"""
# manifest {{
#   description = 'A nexflow pipeline for fg single cell joint qc' 
#   mainScript = 'nf_wf_fg_singlecell_joint.nf'
# }}

# profiles {{
#   google_batch_profile {{
#     workDir = 'gs://eila-nextflow-bucket/work/ATAC'
#     includeConfig 'conf/docker-google-batch.config'
#     docker.enabled = true
#     docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
#     executor = 'google-batch'
#     google.project = 'hai-gcp-genomic'
#     google.location = 'us-central1'
#     google.batch.spot = false
#     google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
#   }}   
# }}

# // Define ENV_SYNAPSE_TOKEN as a parameter
# params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_PY='joint_cell_plotting.py'
# params.SC_JOINT_CELL_BARCODE_PLOT_RNA_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na.na'
# params.SC_JOINT_CELL_BARCODE_PLOT_ATAC_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na1.na'
# //Minimum number of UMIs/fragments required for a cell to be plotted
# params.SC_JOINT_CELL_BARCODE_PLOT_REMOVE_LOW_YIELDING_CELLS=5
# params.SC_JOINT_CELL_BARCODE_PLOT_METADATA_FILE='{nf_data_base_path}nf_wf_fg_singlecell/na2.na'
# params.SC_JOINT_CELL_BARCODE_PLOT_MIN_UMIS=5
# params.SC_JOINT_CELL_BARCODE_PLOT_MIN_GENES=5
# params.SC_JOINT_CELL_BARCODE_PLOT_MIN_TSS=5
# //Cutoff for minimum number of ATAC fragments
# params.SC_JOINT_CELL_BARCODE_PLOT_MIN_FRAGS=5
# params.SC_JOINT_CELL_BARCODE_PLOT_PKR='PKR123'
# params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_R='joint_cell_plotting_density.R'

# params.SC_JOINT_CELL_BARCODE_PLOT_HTML_SCRIPT='write_html.py'
# params.SC_JOINT_CELL_BARCODE_PLOT_HTML_ATAC_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na.na'
# params.SC_JOINT_CELL_BARCODE_PLOT_HTML_RNA_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na1.na'
# params.SC_JOINT_CELL_BARCODE_PLOT_HTML_IMAGE_FILES='{nf_data_base_path}nf_wf_fg_singlecell/na2.na'
# params.SC_JOINT_CELL_BARCODE_PLOT_HTML_LOG_FILES='{nf_data_base_path}nf_wf_fg_singlecell/na3.na'
# """)
